In [6]:
import numpy
from pathlib  import Path
from matplotlib import pyplot as plt
import numpy as np
from copy import copy
import rioxarray
import xarray
import a301_lib

import cartopy.crs as ccrs
import cartopy
from pyresample.utils.cartopy import Projection

from affine import Affine
from rioxarray.rioxarray import affine_to_coords

In [7]:
nir_dir = a301_lib.sat_data / "ajb/landsat7/nir"
nir_files = sorted(list(nir_dir.glob("*tiff")))
red_dir = a301_lib.sat_data / "ajb/landsat7/red"
red_files = sorted(list(red_dir.glob("*tiff")))
str(red_files[0])[-19:-5]

'1984-05-16_red'

In [8]:
def clip_ee_to_StHelens(file_list,band_name):
    for file in file_list:

        the_band = rioxarray.open_rasterio(file,masked=True) 

        scaled_band = the_band*the_band.scale_factor
        masked_band = scaled_band
        cartopy_crs = Projection(the_band.rio.crs, the_band.rio.bounds())

        ana_lon, ana_lat = -122.223611, 46.257066 #St Helens analysis region
        con_lat, con_lon = 46.190630, -122.286143 #St Helens control region
        ana_x, ana_y = cartopy_crs.transform_point(ana_lon,ana_lat,ccrs.Geodetic())
        con_x, con_y = cartopy_crs.transform_point(con_lon,con_lat,ccrs.Geodetic())

        #make a 2 km x 2 km box around each point

        ana_ll_x = ana_x - 2000
        ana_ll_y = ana_y - 2000
        ana_ur_x = ana_x + 2000
        ana_ur_y = ana_y + 2000

        con_ll_x = con_x - 2000
        con_ll_y = con_y - 2000
        con_ur_x = con_x + 2000
        con_ur_y = con_y + 2000
        
        ana_bounding_box = ana_ll_x, ana_ll_y, ana_ur_x, ana_ur_y
        con_bounding_box = con_ll_x, con_ll_y, con_ur_x, con_ur_y

        ana = masked_band.rio.clip_box(*ana_bounding_box)
        con = masked_band.rio.clip_box(*con_bounding_box)

        orig = the_band.rio.transform()
        a,b,d,e = orig.a, orig.b, orig.d, orig.e
        c = ana_ll_x  # upper left x
        f = ana_ur_y  # uper left y

        ana_new_transform = Affine(a,b,c,d,e,f)
        con_new_transform = Affine(a,b,c,d,e,f)

        ana_band, ana_height, ana_width = ana.data.shape
        ana_coords = affine_to_coords(ana_new_transform,ana_width,ana_height)

        con_band, con_height, con_width = con.data.shape
        con_coords = affine_to_coords(con_new_transform,con_width,con_height)

        ana_clipped_ds=xarray.DataArray(ana.data,coords=ana_coords,
                                       dims=ana.dims,
                                       attrs=the_band.attrs)
        
        con_clipped_ds=xarray.DataArray(con.data,coords=con_coords,
                                        dims=con.dims,
                                        attrs=the_band.attrs)

        ana_clipped_ds.rio.write_crs(the_band.rio.crs, inplace=True)
        ana_clipped_ds.rio.write_transform(ana_new_transform, inplace=True)

        con_clipped_ds.rio.write_crs(the_band.rio.crs, inplace=True)
        con_clipped_ds.rio.write_transform(con_new_transform, inplace=True)

        file_name = str(file)[-19:-5]
        ana_outfile = a301_lib.sat_data / f"ajb/landsatee/clipped/ana/{band_name}/ana_clipped_{file_name}.tiff"
        con_outfile = a301_lib.sat_data / f"ajb/landsatee/clipped/con/{band_name}/con_clipped_{file_name}.tiff"

        ana_clipped_ds.rio.to_raster(ana_outfile)
        con_clipped_ds.rio.to_raster(con_outfile)


In [9]:
clip_ee_to_StHelens(nir_files,"nir")
clip_ee_to_StHelens(red_files,"red")